In [1]:
import pandas as pd
import mysql.connector as sql
from geopy.geocoders import Nominatim
import geopy.geocoders

pd.options.display.max_colwidth = 200

In [2]:
indices = ['hoou','oerinfo','hhu','openrub','digill','zoerr', 'tibav', 'oncampus']

col_names = [
    "name",
    "about",
    "author",
    "publisher",
    "inLanguage",
    "accessibilityAPI",
    "accessibilityControl",
    "accessibilityFeature",
    "accessibilityHazard",
    "license",
    "timeRequired",
    "educationalRole",
    "alignmentType",
    "educationalFramework",
    "targetDescription",
    "targetName",
    "targetURL",
    "educationalUse",
    "typicalAgeRange",
    "interactivityType",
    "learningResourceType",
    "date_published",
    "url",
    "thumbnail",
    "tags",
    "project",
    "source",
    "spider",
    "date_scraped"
]

df = {}

def make_dataframes():
    for index in indices:
        db_connection = sql.connect(
        host='localhost', 
        database='oerhoernchen_db', 
        user='oerhoernchen', 
        password='oerhoernchenpw')
        db_cursor = db_connection.cursor()
        db_cursor.execute('SELECT * FROM ' + index + '_tb')
        table_rows = db_cursor.fetchall()
        df[index] = pd.DataFrame(table_rows, columns=col_names)

make_dataframes()

In [19]:
# Geo stuff
geolocator = Nominatim(user_agent="oercrawler")
location_digill = geolocator.geocode("Ruhr-Universität Bochum")
location_hhu = geolocator.geocode("Heinrich-Heine-Universität Düsseldorf")
location_hoou = geolocator.geocode("Hamburg Online University")
location_oerinfo = geolocator.geocode("DIPF Leibniz-Institut für Bildungsforschung und Bildungsinformation")
location_oncampus = geolocator.geocode("Fachhochschule Lübeck")
location_openrub = geolocator.geocode("Ruhr Universität Bochum")
location_tibav = geolocator.geocode("TIB Hannover")
location_zoerr = geolocator.geocode("Universität Tübingen")

In [20]:
def add_location(row):
    if row['source'] == "HOOU":
        address = location_hoou.address 
        lat = location_hoou.latitude
        lon = location_hoou.longitude     
        return address, lat, lon
    elif row['source'] == "digiLL":
        address = location_digill.address 
        lat = location_digill.latitude
        lon = location_digill.longitude
        return address, lat, lon
    elif row['source'] == "HHU-Mediathek":
        address = location_hhu.address 
        lat = location_hhu.latitude
        lon = location_hhu.longitude
        return address, lat, lon
    elif row['source'] == "OERinfo":
        address = location_oerinfo.address 
        lat = location_oerinfo.latitude
        lon = location_oerinfo.longitude 
        return address, lat, lon
    elif row['source'] == "ONCAMPUS":
        address = location_oncampus.address 
        lat = location_oncampus.latitude
        lon = location_oncampus.longitude 
        return address, lat, lon
    elif row['source'] == "OpenRub":
        address = location_openrub.address 
        lat = location_openrub.latitude
        lon = location_openrub.longitude 
        return address, lat, lon
    elif row['source'] == "TIB-AV-Portal":
        address = location_tibav.address 
        lat = location_tibav.latitude
        lon = location_tibav.longitude 
        return address, lat, lon
    elif row['source'] == "ZOERR":
        address = location_zoerr.address 
        lat = location_zoerr.latitude
        lon = location_zoerr.longitude 
        return address, lat, lon

In [21]:
def write_location_to_df():
    for index in indices:
        df[index]['location'] = df[index].apply(add_location, axis=1)
        df[index]['lat_lon'] = df[index]['location'].apply(lambda x: (x[-2], x[-1]))
        df[index].drop(columns=['location'], inplace=True)
        df[index]['lat_lon'] = df[index]['lat_lon'].astype(str)
        df[index]['lat_lon'] = df[index]['lat_lon'].str.replace("(", "").str.replace(")", "")
write_location_to_df()

In [1]:
# Import dataframe into MySQL
import sqlalchemy
database_username = 'root'
database_password = 'my-secret-pw'
database_ip       = 'localhost:3306'
database_name     = 'oerhoernchen_db'
engine = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.format(database_username, database_password, database_ip, database_name))

In [23]:
for index in indices:
    df[index].to_sql(name=index + "_tb_edited", con=engine, index=False, if_exists='replace')

In [6]:
# How to read in with sqlalchemy
df2 = pd.read_sql_table("hhu_tb", engine)
df2

NameError: name 'engine' is not defined

In [2]:
import pandas as pd


In [3]:
df = pd.read_sql_table("indices", engine)

In [4]:
df

,name,url,read
0,HOOU,www.hoou.de/sitemap.xml,1
1,oer_oerinfo,www.oerinfo.de/sitemap.xml,1


In [5]:
SITEMAPS = [{'name': 'HOOU', 'url': 'www.hoou.de/sitemap.xml', 'read': 1}, {'name': 'oer_oerinfo', 'url': 'www.oerinfo.de/sitemap.xml', 'read': 1}]

In [6]:
df_list = pd.DataFrame(SITEMAPS)

In [7]:
df_list

,name,url,read
0,HOOU,www.hoou.de/sitemap.xml,1
1,oer_oerinfo,www.oerinfo.de/sitemap.xml,1
